In [1]:
import os, random
from pydub import AudioSegment
from xml.dom import minidom
import numpy as np
import pandas as pd
from pysndfx import AudioEffectsChain
from librosa import load
import shutil

In [2]:
f = open("D://Background_noise//meta.txt","r")
match_noise_to_type = []
content = f.readlines()

for line in content:
    y = line.split('	')
    audio_filename = y[0][6:len(y[0])]
    audio_category = y[1][0:len(y[1])-1].replace('/', '_')
    
    match_noise_to_type.append((audio_filename, audio_category))

noise_dict = dict(match_noise_to_type)
#print(noise_dict)

In [7]:
h_directory = 'D://Datasets//TRAINING//Happy//'
a_directory = 'D://Datasets//TRAINING//Angry//'
n_directory = 'D://Datasets//TRAINING//Neutral//'
s_directory = 'D://Datasets//TRAINING//Sad//'
o_directory = 'D://Datasets//TRAINING//Other//'

h_test = 'D://Datasets//TRAINING//Happy_test//'
a_test = 'D://Datasets//TRAINING//Angry_test//'
n_test = 'D://Datasets//TRAINING//Neutral_test//'
s_test = 'D://Datasets//TRAINING//Sad_test//'
o_test = 'D://Datasets//TRAINING//Other_test//'

h_EMO = 'D://Datasets//EMO-DB//wav//Happy//'
a_EMO = 'D://Datasets//EMO-DB//wav//Angry//'
n_EMO = 'D://Datasets//EMO-DB//wav//Neutral//'
s_EMO = 'D://Datasets//EMO-DB//wav//Sad//'
o_EMO = 'D://Datasets//EMO-DB//wav//Other//'

background_noise = 'D://Background_noise//TUT2016//'
background_noise_test = 'D://Background_noise//noise_test//'
background_noise_all = 'D://Background_noise//noise_all//'
background_noise_home = 'D://Background_noise//noise_home//'

emotion_train_folders = [h_directory, a_directory, n_directory, s_directory, o_directory]
emotion_test_folders = [h_test, a_test, n_test, s_test, o_test]

In [8]:
def rename_emotion_wavs(emotion_directory):
    if 'Happy' in emotion_directory:
        word = 'Happy'
    elif 'Angry' in emotion_directory:
        word = 'Angry'
    elif 'Neutral' in emotion_directory:
        word = 'Neutral'
    elif 'Sad' in emotion_directory:
        word = 'Sad'
    else:
        word = 'Other'
    
    index = 0
    for file in os.listdir(emotion_directory):
        src = emotion_directory + file
        dst = emotion_directory + word + str(index) + '.wav'
        try:
            os.rename(src, dst)
        except WindowsError:
            os.remove(dst)
            os.rename(src, dst)
        index += 1
        print(dst)
        
#rename_emotion_wavs(h_directory)
#rename_emotion_wavs(a_directory)
#rename_emotion_wavs(n_directory)
#rename_emotion_wavs(s_directory)
#rename_emotion_wavs(o_directory)

In [9]:
def rename_noise_wavs(noise_directory):
    index = 0
    for noise in os.listdir(noise_directory):
        noise_type = noise_dict.get(noise)
        new_noise_name = noise_type + '_' + str(index)
        src = noise_directory + noise
        dst = noise_directory + new_noise_name + '.wav'
        os.rename(src, dst) 
        index += 1

        print(dst)

#rename_noise_wavs(background_noise_all)

In [10]:
def delete_directory(path):
    shutil.rmtree(path, ignore_errors=True)

In [11]:
def partition_directory(path, new_path, percent):
    
    if not os.path.exists(new_path):
        os.mkdir(new_path)
    else:
        delete_directory(new_path)
        os.makdir(new_path)
        
    total = len(os.listdir(path))
    test_number = int(total * percent)
    test_set = np.random.permutation(total)[0:test_number]
    
    for index in test_set:
        try:
            if os.listdir(path)[index].endswith('.wav'):
                original_location = path + os.listdir(path)[index]
                new_location = new_path + os.listdir(path)[index]
                os.rename(original_location, new_location)
                print(new_location)
        except:
            pass

    
    print('# of training data + # of testing data = ' + str(total))
    print(str(percent) + ' of the data is in training set')
    print('# of training data = ' + str(test_set.shape))


In [13]:
def pad_silence_per_file(emotionfile, newSoundFile):
    
    emotionsound = AudioSegment.from_wav(emotionfile)
    file_duration = emotionsound.duration_seconds * 1000
    silence = AudioSegment.silent(duration=5000)
    threshold = 5000 - file_duration
    if threshold > 0:
        overlay_start = np.random.randint(0, threshold)
    else:
        emotionsound = emotionsound[0:5000]
        overlay_start = 0
    
    newSound = silence.overlay(emotionsound, position=overlay_start)
    
    newSound=newSound[0:5000]
    newSound.export(newSoundFile, format='wav')  ### save the new generated file in a folder

In [14]:
def pad_silence_per_folder(directory, extension):
    for file in os.listdir(directory):
        if file.endswith(extension) and file[0] != '.':
            newSoundFile = directory + file
            pad_silence_per_file(newSoundFile, newSoundFile)
            print(newSoundFile)

In [16]:
'''
pad_silence_per_folder(path, '.wav')

for path in emotion_test_folders:
    pad_silence_per_folder(path, '.wav')
'''
#pad_silence_per_folder(h_directory, '.wav')
#pad_silence_per_folder(h_test, '.wav')

"\npad_silence_per_folder(path, '.wav')\n\nfor path in emotion_test_folders:\n    pad_silence_per_folder(path, '.wav')\n"

In [17]:
def change_amplitude(emotionfile, d1, newSoundFile, d2):
    
    if d1 <= d2:
        sound = AudioSegment.from_file(emotionfile) - np.random.randint(0, (6 * d2/d1 - 1))
        sound.export(newSoundFile, format='wav')  ### save the new generated file in a folder
    else:
        print('Invalid distance parameters. d1 should be <= d2.')

In [18]:
def change_amplitude_range(emotionfile, newSoundFile, threshold):
    #amount = np.random.randint(0, threshold)
    amount = random.uniform(0, threshold)
    #print('Deamplify ' + str(emotionfile) + ' by ' + str(amount) + ' db.')
    sound = AudioSegment.from_file(emotionfile) - amount
    sound.export(newSoundFile, format='wav')  ### save the new generated file in a folder
    return amount

In [19]:
'''
def deamplify_per_folder(directory):
    for file in os.listdir(directory):
        if file.endswith('.wav'):
            soundFile = directory + file
            newSoundFile = directory + 'deamplified_' + file
            change_amplitude_range(soundFile, newSoundFile, 12)
'''

"\ndef deamplify_per_folder(directory):\n    for file in os.listdir(directory):\n        if file.endswith('.wav'):\n            soundFile = directory + file\n            newSoundFile = directory + 'deamplified_' + file\n            change_amplitude_range(soundFile, newSoundFile, 12)\n"

In [20]:
'''
for directory in emotion_train_folders:
    deamplify_per_folder(directory)
    
for directory in emotion_test_folders:
    deamplify_per_folder(directory)
'''

'\nfor directory in emotion_train_folders:\n    deamplify_per_folder(directory)\n    \nfor directory in emotion_test_folders:\n    deamplify_per_folder(directory)\n'

In [21]:
def add_noise_per_file(emotionfile, bgnoise, newSoundFile):
    
    emotionsound = AudioSegment.from_wav(emotionfile)
    emotion_duration = emotionsound.duration_seconds * 1000
    noise = AudioSegment.from_wav(bgnoise)
    noise_duration = noise.duration_seconds * 1000
    
    threshold = noise_duration - emotion_duration
    
    if threshold > 0:
        overlay_start = np.random.randint(0, threshold)
    else:
        overlay_start = 0
        
    targeted_chunk = noise[overlay_start:overlay_start + emotion_duration]
    newSound = emotionsound.overlay(targeted_chunk, position=0)
    newSound=newSound[0:5000]
    newSound.export(newSoundFile, format='wav')  ### save the new generated file in a folder

In [22]:
'''
def add_noise_per_folder(directory, extension, noise_directory):
    for file in os.listdir(directory):
        if file.endswith(extension) and not file[1] == '_':
            for i in range(0, 3):
                soundFile = directory + file
                newSoundFile = directory + str(i) + '_' + file
                random_noise = noise_directory + random.choice(os.listdir(noise_directory))

                add_noise_per_file(soundFile, random_noise, newSoundFile)
                print(newSoundFile)
'''

"\ndef add_noise_per_folder(directory, extension, noise_directory):\n    for file in os.listdir(directory):\n        if file.endswith(extension) and not file[1] == '_':\n            for i in range(0, 3):\n                soundFile = directory + file\n                newSoundFile = directory + str(i) + '_' + file\n                random_noise = noise_directory + random.choice(os.listdir(noise_directory))\n\n                add_noise_per_file(soundFile, random_noise, newSoundFile)\n                print(newSoundFile)\n"

In [24]:
def add_noise_and_deamplify_per_folder(directory, extension, noise_directory):
    for file in os.listdir(directory):
        if file.endswith(extension) and not file[1] == '_':
            for i in range(0, 2):
                soundFile = directory + file
                amount = change_amplitude_range(soundFile, soundFile, 12)
                noise = random.choice(os.listdir(noise_directory))
                random_noise = noise_directory + noise
                newSoundFile = directory + 'deamp_' + str(amount) + '_noise_' + noise[:len(noise)-5] + '_' + file
                add_noise_per_file(soundFile, random_noise, newSoundFile)
                print(newSoundFile)

            

In [26]:
add_noise_and_deamplify_per_folder(h_directory, 'wav', background_noise_all)
add_noise_and_deamplify_per_folder(a_directory, 'wav', background_noise_all)
add_noise_and_deamplify_per_folder(n_directory, 'wav', background_noise_all)
add_noise_and_deamplify_per_folder(s_directory, 'wav', background_noise_all)
add_noise_and_deamplify_per_folder(o_directory, 'wav', background_noise_all)

D://Datasets//TRAINING//Happy//deamp_5.272859028810952_noise_home_80_Happy374.wav
D://Datasets//TRAINING//Happy//deamp_3.8630051658404256_noise_home_82_Happy374.wav
D://Datasets//TRAINING//Happy//deamp_8.088740604428956_noise_home_25_Happy375.wav
D://Datasets//TRAINING//Happy//deamp_1.133560230681161_noise_home_25_Happy375.wav
D://Datasets//TRAINING//Happy//deamp_11.111324824243262_noise_home_2_Happy376.wav
D://Datasets//TRAINING//Happy//deamp_0.6957744296403696_noise_home_82_Happy376.wav
D://Datasets//TRAINING//Happy//deamp_4.441770386748357_noise_home_22_Happy377.wav
D://Datasets//TRAINING//Happy//deamp_5.818458684522847_noise_home_83_Happy377.wav
D://Datasets//TRAINING//Happy//deamp_8.65541213343424_noise_home_88_Happy378.wav
D://Datasets//TRAINING//Happy//deamp_7.256412044975331_noise_home_24_Happy378.wav
D://Datasets//TRAINING//Happy//deamp_0.2617055772021546_noise_home_81_Happy379.wav
D://Datasets//TRAINING//Happy//deamp_4.602333392315133_noise_home_25_Happy379.wav
D://Datasets//

D://Datasets//TRAINING//Happy//deamp_4.577109131411807_noise_home_80_Happy424.wav
D://Datasets//TRAINING//Happy//deamp_3.03582427298817_noise_home_24_Happy424.wav
D://Datasets//TRAINING//Happy//deamp_4.5773059806232705_noise_home_83_Happy425.wav
D://Datasets//TRAINING//Happy//deamp_10.860011331647723_noise_home_83_Happy425.wav
D://Datasets//TRAINING//Happy//deamp_0.17377161255966644_noise_home_26_Happy426.wav
D://Datasets//TRAINING//Happy//deamp_8.53090270634851_noise_home_83_Happy426.wav
D://Datasets//TRAINING//Happy//deamp_0.5855149423886372_noise_home_81_Happy427.wav
D://Datasets//TRAINING//Happy//deamp_1.0245756767619554_noise_home_81_Happy427.wav
D://Datasets//TRAINING//Happy//deamp_3.7027925737910548_noise_home_26_Happy428.wav
D://Datasets//TRAINING//Happy//deamp_5.992029487563812_noise_home_88_Happy428.wav
D://Datasets//TRAINING//Happy//deamp_0.7349168268463533_noise_home_88_Happy429.wav
D://Datasets//TRAINING//Happy//deamp_9.209433154847023_noise_home_23_Happy429.wav
D://Datase

D://Datasets//TRAINING//Happy//deamp_8.726323813358357_noise_home_81_Happy475.wav
D://Datasets//TRAINING//Happy//deamp_6.694634582107495_noise_home_81_Happy475.wav
D://Datasets//TRAINING//Happy//deamp_10.106009260731433_noise_home_80_Happy476.wav
D://Datasets//TRAINING//Happy//deamp_3.1421185773606495_noise_home_88_Happy476.wav
D://Datasets//TRAINING//Happy//deamp_5.300938408791185_noise_home_23_Happy477.wav
D://Datasets//TRAINING//Happy//deamp_4.70929123419271_noise_home_81_Happy477.wav
D://Datasets//TRAINING//Happy//deamp_1.5783029191960116_noise_home_26_Happy479.wav
D://Datasets//TRAINING//Happy//deamp_5.9416001922179795_noise_home_80_Happy479.wav
D://Datasets//TRAINING//Happy//deamp_2.758332041514304_noise_home_81_Happy480.wav
D://Datasets//TRAINING//Happy//deamp_2.0009488657802117_noise_home_81_Happy480.wav
D://Datasets//TRAINING//Happy//deamp_2.134938975162757_noise_home_24_Happy481.wav
D://Datasets//TRAINING//Happy//deamp_5.484966224404577_noise_home_88_Happy481.wav
D://Datasets

D://Datasets//TRAINING//Happy//deamp_11.395173884798531_noise_home_81_Happy528.wav
D://Datasets//TRAINING//Happy//deamp_2.1781548980289314_noise_home_80_Happy528.wav
D://Datasets//TRAINING//Happy//deamp_5.862887683654172_noise_home_81_Happy529.wav
D://Datasets//TRAINING//Happy//deamp_6.566976640145835_noise_home_81_Happy529.wav
D://Datasets//TRAINING//Happy//deamp_4.696462417013192_noise_home_81_Happy530.wav
D://Datasets//TRAINING//Happy//deamp_11.363626328368404_noise_home_88_Happy530.wav
D://Datasets//TRAINING//Happy//deamp_6.971030068156663_noise_home_2_Happy531.wav
D://Datasets//TRAINING//Happy//deamp_8.51955228206366_noise_home_22_Happy531.wav
D://Datasets//TRAINING//Happy//deamp_0.27386348157545415_noise_home_25_Happy532.wav
D://Datasets//TRAINING//Happy//deamp_1.9556379091187446_noise_home_26_Happy532.wav
D://Datasets//TRAINING//Happy//deamp_9.799505994741144_noise_home_80_Happy534.wav
D://Datasets//TRAINING//Happy//deamp_8.20539843472522_noise_home_81_Happy534.wav
D://Datasets/

D://Datasets//TRAINING//Happy//deamp_6.051855336264561_noise_home_25_Happy580.wav
D://Datasets//TRAINING//Happy//deamp_9.079835895250035_noise_home_81_Happy580.wav
D://Datasets//TRAINING//Happy//deamp_2.896809954403954_noise_home_82_Happy581.wav
D://Datasets//TRAINING//Happy//deamp_3.144913196957109_noise_home_80_Happy581.wav
D://Datasets//TRAINING//Happy//deamp_2.2853424477113227_noise_home_2_Happy582.wav
D://Datasets//TRAINING//Happy//deamp_5.673815144612284_noise_home_23_Happy582.wav
D://Datasets//TRAINING//Happy//deamp_1.6176483408310531_noise_home_23_Happy583.wav
D://Datasets//TRAINING//Happy//deamp_9.262127202566084_noise_home_80_Happy583.wav
D://Datasets//TRAINING//Happy//deamp_5.773708163263411_noise_home_80_Happy584.wav
D://Datasets//TRAINING//Happy//deamp_2.2908012938967146_noise_home_81_Happy584.wav
D://Datasets//TRAINING//Happy//deamp_11.291291633062908_noise_home_26_Happy585.wav
D://Datasets//TRAINING//Happy//deamp_2.5900984792643857_noise_home_82_Happy585.wav
D://Datasets

...

D://Datasets//TRAINING//Other//deamp_9.16005149448723_noise_home_2_Other2384.wav
D://Datasets//TRAINING//Other//deamp_7.952906095220252_noise_home_80_Other2385.wav
D://Datasets//TRAINING//Other//deamp_6.2897943215759025_noise_home_25_Other2385.wav
D://Datasets//TRAINING//Other//deamp_5.929836376456336_noise_home_82_Other2386.wav
D://Datasets//TRAINING//Other//deamp_2.458277833273301_noise_home_2_Other2386.wav
D://Datasets//TRAINING//Other//deamp_8.095302265530982_noise_home_82_Other2387.wav
D://Datasets//TRAINING//Other//deamp_6.930935400199191_noise_home_88_Other2387.wav
D://Datasets//TRAINING//Other//deamp_1.61503828478364_noise_home_80_Other2388.wav
D://Datasets//TRAINING//Other//deamp_10.848140329865323_noise_home_27_Other2388.wav
D://Datasets//TRAINING//Other//deamp_2.9131016656629845_noise_home_23_Other2389.wav
D://Datasets//TRAINING//Other//deamp_11.863767859099681_noise_home_88_Other2389.wav
D://Datasets//TRAINING//Other//deamp_5.997377052595093_noise_home_2_Other2390.wav
D://D

D://Datasets//TRAINING//Other//deamp_7.405805475073315_noise_home_23_Other2434.wav
D://Datasets//TRAINING//Other//deamp_8.104858120744314_noise_home_81_Other2434.wav
D://Datasets//TRAINING//Other//deamp_5.318163392093866_noise_home_81_Other2435.wav
D://Datasets//TRAINING//Other//deamp_9.008368032371557_noise_home_88_Other2435.wav
D://Datasets//TRAINING//Other//deamp_10.034075118755013_noise_home_83_Other2436.wav
D://Datasets//TRAINING//Other//deamp_10.020127983670447_noise_home_26_Other2436.wav
D://Datasets//TRAINING//Other//deamp_9.586751320041582_noise_home_2_Other2437.wav
D://Datasets//TRAINING//Other//deamp_3.963011051244901_noise_home_82_Other2437.wav
D://Datasets//TRAINING//Other//deamp_2.0933237860255884_noise_home_23_Other2438.wav
D://Datasets//TRAINING//Other//deamp_5.771384308182563_noise_home_81_Other2438.wav
D://Datasets//TRAINING//Other//deamp_11.862749921231737_noise_home_24_Other2439.wav
D://Datasets//TRAINING//Other//deamp_7.223475057425031_noise_home_26_Other2439.wav
D

D://Datasets//TRAINING//Other//deamp_0.9889887752668427_noise_home_25_Other2483.wav
D://Datasets//TRAINING//Other//deamp_8.671582488444045_noise_home_83_Other2484.wav
D://Datasets//TRAINING//Other//deamp_2.348562250027581_noise_home_80_Other2484.wav
D://Datasets//TRAINING//Other//deamp_1.9095112662278302_noise_home_82_Other2485.wav
D://Datasets//TRAINING//Other//deamp_7.051895372659532_noise_home_23_Other2485.wav
D://Datasets//TRAINING//Other//deamp_10.859007119798818_noise_home_23_Other2486.wav
D://Datasets//TRAINING//Other//deamp_8.79855206003416_noise_home_82_Other2486.wav
D://Datasets//TRAINING//Other//deamp_6.988209200071783_noise_home_80_Other2487.wav
D://Datasets//TRAINING//Other//deamp_1.2838691003537588_noise_home_81_Other2487.wav
D://Datasets//TRAINING//Other//deamp_5.553484970023457_noise_home_82_Other2488.wav
D://Datasets//TRAINING//Other//deamp_5.084829570139579_noise_home_80_Other2488.wav
D://Datasets//TRAINING//Other//deamp_0.47623349714203034_noise_home_26_Other2489.wav

D://Datasets//TRAINING//Other//deamp_1.1507007879085456_noise_home_26_Other2533.wav
D://Datasets//TRAINING//Other//deamp_10.48762097288672_noise_home_23_Other2533.wav
D://Datasets//TRAINING//Other//deamp_0.05246579948193064_noise_home_82_Other2534.wav
D://Datasets//TRAINING//Other//deamp_8.234003117737625_noise_home_23_Other2534.wav
D://Datasets//TRAINING//Other//deamp_10.745698352420993_noise_home_25_Other2535.wav
D://Datasets//TRAINING//Other//deamp_1.6331380352985554_noise_home_22_Other2535.wav
D://Datasets//TRAINING//Other//deamp_0.8577732220419905_noise_home_26_Other2536.wav
D://Datasets//TRAINING//Other//deamp_4.654863786633973_noise_home_2_Other2536.wav
D://Datasets//TRAINING//Other//deamp_9.062958073155501_noise_home_88_Other2537.wav
D://Datasets//TRAINING//Other//deamp_2.537671080231982_noise_home_80_Other2537.wav
D://Datasets//TRAINING//Other//deamp_9.826192025137468_noise_home_27_Other2538.wav
D://Datasets//TRAINING//Other//deamp_1.9798028954112592_noise_home_2_Other2538.wav

CouldntDecodeError: Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

b'ffmpeg version 2.7 Copyright (c) 2000-2015 the FFmpeg developers\r\n  built with gcc 4.9.2 (GCC)\r\n  configuration: --enable-gpl --enable-version3 --disable-w32threads --enable-avisynth --enable-bzlib --enable-fontconfig --enable-frei0r --enable-gnutls --enable-iconv --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libdcadec --enable-libfreetype --enable-libgme --enable-libgsm --enable-libilbc --enable-libmodplug --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-librtmp --enable-libschroedinger --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvo-aacenc --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs --enable-libxvid --enable-lzma --enable

In [ ]:
percent = 0.2
partition_directory(h_directory, h_test, percent)
partition_directory(a_directory, a_test, percent)
partition_directory(n_directory, n_test, percent)
partition_directory(s_directory, s_test, percent)
partition_directory(o_directory, o_test, percent)

In [ ]:
add_noise_and_deamplify_per_folder(h_test, '.wav', background_noise_test)

In [ ]:
add_noise_and_deamplify_per_folder(a_test, '.wav', background_noise_test)

In [ ]:
add_noise_and_deamplify_per_folder(n_test, '.wav', background_noise_test)

In [ ]:
add_noise_and_deamplify_per_folder(s_test, '.wav', background_noise_test)

In [ ]:
add_noise_and_deamplify_per_folder(o_test, '.wav', background_noise_test)

In [ ]:
# AudioEffectsChain().reverb()(infile, outfile)